In [1]:
import numpy as np
import cv2
from collections import deque

In [2]:
# Define the upper and lower boundaries for a color to be considered "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Initialize deques to store different colors in different arrays
bpoints = [deque(maxlen=512)]
gpoints = [deque(maxlen=512)]
rpoints = [deque(maxlen=512)]
ypoints = [deque(maxlen=512)]

# Initialize an index variable for each of the colors 
bindex = 0
gindex = 0
rindex = 0
yindex = 0

# Just a handy array and an index variable to get the color-of-interest on the go
# Blue, Green, Red, Yellow respectively
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)] 
colorIndex = 0

In [3]:
# Create a blank white image
paintWindow = np.zeros((471,636,3)) + 255

# Draw buttons like colored rectangles on the white image
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), colors[0], -1)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), colors[1], -1)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), colors[2], -1)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), colors[3], -1)

# Label the rectanglular boxes drawn on the image
cv2.putText(paintWindow, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)

# Create a window to display the above image (later)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

In [ ]:
# Load the video
camera = cv2.VideoCapture(0)

# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Check to see if we have reached the end of the video (useful when input is a video file not a live video stream)
    if not grabbed:
        break    
    
    # Add the same paint interface to the camera feed captured through the webcam (for ease of usage)
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours in the image
    (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    
    # Check to see if any contours (blue stuff) were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case a circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
    # Check to see if any contours (blue stuff) were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case a circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
    # Draw lines of all the colors (Blue, Green, Red and Yellow)
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    # Show the frame and the paintWindow image
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintWindow)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# Cleanup code
camera.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
from collections import deque

# Define the upper and lower boundaries for a color to be considered "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Initialize deques to store different colors in different arrays
bpoints = [deque(maxlen=512)]
gpoints = [deque(maxlen=512)]
rpoints = [deque(maxlen=512)]
ypoints = [deque(maxlen=512)]

# Initialize an index variable for each of the colors 
bindex = 0
gindex = 0
rindex = 0
yindex = 0

# Just a handy array and an index variable to get the color-of-interest on the go
# Blue, Green, Red, Yellow respectively
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)] 
colorIndex = 0

# Load the video
camera = cv2.VideoCapture(0)

# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Check to see if we have reached the end of the video (useful when input is a video file not a live video stream)
    if not grabbed:
        break    
    
    # Add the same paint interface to the camera feed captured through the webcam (for ease of usage)
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours in the image
    (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    
    # Check to see if any contours (blue stuff) were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case a circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
    # Check to see if any contours (blue stuff) were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case a circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
    # Draw lines of all the colors (Blue, Green, Red and Yellow)
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    # Show the frame and the paintWindow image
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintWindow)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# Cleanup code
camera.release()
cv2.destroyAllWindows()

In [8]:
from tkinter import *


class Paint(Frame):

    def __init__(self, parent):
        Frame.__init__(self, parent)

        self.parent = parent
        self.color = "black"
        self.brush_size = 2
        self.setUI()

    def set_color(self, new_color):
        self.color = new_color

    def set_brush_size(self, new_size):
        self.brush_size = new_size

    def draw(self, event):
        self.canv.create_oval(event.x - self.brush_size,
                              event.y - self.brush_size,
                              event.x + self.brush_size,
                              event.y + self.brush_size,
                              fill=self.color, outline=self.color)

    def setUI(self):

        self.parent.title("Pythonicway PyPaint")  # Устанавливаем название окна
        self.pack(fill=BOTH, expand=1)  # Размещаем активные элементы на родительском окне

        self.columnconfigure(6, weight=1) # Даем седьмому столбцу возможность растягиваться, благодаря чему кнопки не будут разъезжаться при ресайзе
        self.rowconfigure(2, weight=1) # То же самое для третьего ряда

        self.canv = Canvas(self, bg="white")  # Создаем поле для рисования, устанавливаем белый фон
        self.canv.grid(row=2, column=0, columnspan=7,
                       padx=5, pady=5, sticky=E+W+S+N)  # Прикрепляем канвас методом grid. Он будет находится в 3м ряду, первой колонке, и будет занимать 7 колонок, задаем отступы по X и Y в 5 пикселей, и заставляем растягиваться при растягивании всего окна
        self.canv.bind("<B1-Motion>", self.draw) # Привязываем обработчик к канвасу. <B1-Motion> означает "при движении зажатой левой кнопки мыши" вызывать функцию draw

        color_lab = Label(self, text="Color: ") # Создаем метку для кнопок изменения цвета кисти
        color_lab.grid(row=0, column=0, padx=6) # Устанавливаем созданную метку в первый ряд и первую колонку, задаем горизонтальный отступ в 6 пикселей

        red_btn = Button(self, text="Red", width=10,
                         command=lambda: self.set_color("red")) # Создание кнопки:  Установка текста кнопки, задание ширины кнопки (10 символов), функция вызываемая при нажатии кнопки.
        red_btn.grid(row=0, column=1) # Устанавливаем кнопку

        # Создание остальных кнопок повторяет ту же логику, что и создание
        # кнопки установки красного цвета, отличаются лишь аргументы.

        green_btn = Button(self, text="Green", width=10,
                           command=lambda: self.set_color("green"))
        green_btn.grid(row=0, column=2)

        blue_btn = Button(self, text="Blue", width=10,
                          command=lambda: self.set_color("blue"))
        blue_btn.grid(row=0, column=3)

        black_btn = Button(self, text="Black", width=10,
                           command=lambda: self.set_color("black"))
        black_btn.grid(row=0, column=4)

        white_btn = Button(self, text="White", width=10,
                           command=lambda: self.set_color("white"))
        white_btn.grid(row=0, column=5)

        clear_btn = Button(self, text="Clear all", width=10,
                           command=lambda: self.canv.delete("all"))
        clear_btn.grid(row=0, column=6, sticky=W)

        size_lab = Label(self, text="Brush size: ")
        size_lab.grid(row=1, column=0, padx=5)
        one_btn = Button(self, text="Two", width=10,
                         command=lambda: self.set_brush_size(2))
        one_btn.grid(row=1, column=1)

        two_btn = Button(self, text="Five", width=10,
                         command=lambda: self.set_brush_size(5))
        two_btn.grid(row=1, column=2)

        five_btn = Button(self, text="Seven", width=10,
                          command=lambda: self.set_brush_size(7))
        five_btn.grid(row=1, column=3)

        seven_btn = Button(self, text="Ten", width=10,
                           command=lambda: self.set_brush_size(10))
        seven_btn.grid(row=1, column=4)

        ten_btn = Button(self, text="Twenty", width=10,
                         command=lambda: self.set_brush_size(20))
        ten_btn.grid(row=1, column=5)

        twenty_btn = Button(self, text="Fifty", width=10,
                            command=lambda: self.set_brush_size(50))
        twenty_btn.grid(row=1, column=6, sticky=W)


def main():
    root = Tk()
    root.geometry("500x500+30+30")
    app = Paint(root)
    root.mainloop()


if __name__ == '__main__':
    main()

In [19]:
from tkinter import *
import cv2



class Paint(Frame):

    def __init__(self, parent):
        Frame.__init__(self, parent)

        self.parent = parent
        self.color = "green"
        self.brush_size = 5
        self.setUI()

    def set_color(self, new_color):
        self.color = new_color

    def set_brush_size(self, new_size):
        self.brush_size = new_size

    def draw(self, event):
        self.canv.create_oval(event.x - self.brush_size,
                              event.y - self.brush_size,
                              event.x + self.brush_size,
                              event.y + self.brush_size,
                              fill=self.color, outline=self.color)

    def setUI(self):

        self.parent.title("Бильярд промо")  # Устанавливаем название окна
        self.pack(fill=BOTH, expand=1)  # Размещаем активные элементы на родительском окне

        self.columnconfigure(6, weight=1) # Даем седьмому столбцу возможность растягиваться, благодаря чему кнопки не будут разъезжаться при ресайзе
        self.rowconfigure(2, weight=1) # То же самое для третьего ряда
        
        bg_image = PhotoImage(file="Backgroung.png")
        
        self.canv = Canvas(self, bg=bg_image)  # Создаем поле для рисования, устанавливаем белый фон
        self.canv.grid(row=2, column=0, columnspan=7,
                       padx=5, pady=5, sticky=E+W+S+N)  # Прикрепляем канвас методом grid. Он будет находится в 3м ряду, первой колонке, и будет занимать 7 колонок, задаем отступы по X и Y в 5 пикселей, и заставляем растягиваться при растягивании всего окна
        self.canv.bind("<B1-Motion>", self.draw) # Привязываем обработчик к канвасу. <B1-Motion> означает "при движении зажатой левой кнопки мыши" вызывать функцию draw

        color_lab = Label(self, text="Color: ") # Создаем метку для кнопок изменения цвета кисти
        color_lab.grid(row=0, column=0, padx=6) # Устанавливаем созданную метку в первый ряд и первую колонку, задаем горизонтальный отступ в 6 пикселей

        red_btn = Button(self, text="Red", width=10,
                         command=lambda: self.set_color("red")) # Создание кнопки:  Установка текста кнопки, задание ширины кнопки (10 символов), функция вызываемая при нажатии кнопки.
        red_btn.grid(row=0, column=1) # Устанавливаем кнопку

        # Создание остальных кнопок повторяет ту же логику, что и создание
        # кнопки установки красного цвета, отличаются лишь аргументы.

        green_btn = Button(self, text="Green", width=10,
                           command=lambda: self.set_color("green"))
        green_btn.grid(row=0, column=2)




def main():
    root = Tk()
    root.geometry("500x500+30+30")
    app = Paint(root)
    root.mainloop()


if __name__ == '__main__':
    main()

TclError: unknown color name "pyimage3"

In [1]:
from tkinter import *
from tkinter import messagebox
top = Tk()

C = Canvas(top, bg="blue", height=250, width=300)
filename = PhotoImage(file = "Backgroung.png")
background_label = Label(top, image=filename)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

C.pack()
top.mainloop

<bound method Misc.mainloop of <tkinter.Tk object .>>

In [1]:
from tkinter import *
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk
import tkinter.simpledialog

root = Tk()

#setting up a tkinter canvas
w = Canvas(root, width=500, height=500)
w.pack()

#adding the image
File = askopenfilename(parent=root, initialdir="./",title='Select an image')
original = Image.open(File)
original = original.resize((1000,1000)) #resize image
img = ImageTk.PhotoImage(original)
w.create_image(0, 0, image=img, anchor="nw")

#ask for pressure and temperature extent
xmt = tkinter.simpledialog.askfloat("Temperature", "degrees in x-axis")
ymp = tkinter.simpledialog.askfloat("Pressure", "bars in y-axis")

#ask for real PT values at origin
xc = tkinter.simpledialog.askfloat("Temperature", "Temperature at origin")
yc = tkinter.simpledialog.askfloat("Pressure", "Pressure at origin")

#instruction on 3 point selection to define grid
tkinter.messagebox.showinfo("Instructions", "Click: \n" 
                                            "1) Origin \n"
                                            "2) Temperature end \n"
                                            "3) Pressure end")

# From here on I have no idea how to get it to work...

# Determine the origin by clicking
def getorigin(eventorigin):
    global x0,y0
    x0 = eventorigin.x
    y0 = eventorigin.y
    print(x0,y0)
    w.bind("<Button 1>",getextentx)
#mouseclick event
w.bind("<Button 1>",getorigin)

# Determine the extent of the figure in the x direction (Temperature)
def getextentx(eventextentx):
    global xe
    xe = eventextentx.x
    print(xe)
    w.bind("<Button 1>",getextenty)

# Determine the extent of the figure in the y direction (Pressure)
def getextenty(eventextenty):
    global ye
    ye = eventextenty.y
    print(ye)
    tkinter.messagebox.showinfo("Grid", "Grid is set. You can start picking coordinates.")
    w.bind("<Button 1>",printcoords)

#Coordinate transformation into Pressure-Temperature space
def printcoords(event):
    xmpx = xe-x0
    xm = xmt/xmpx
    ympx = ye-y0
    ym = -ymp/ympx

    #coordinate transformation
    newx = (event.x-x0)*(xm)+xc
    newy = (event.y-y0)*(ym)+yc

    #outputting x and y coords to console
    print (newx,newy)

root.mainloop()

198 194
306
312
88.61111111111111 -47.54237288135593
128.8425925925926 37.7542372881356
40.23148148148148 55.46610169491525
40.23148148148148 56.398305084745765
40.23148148148148 56.398305084745765
40.74074074074074 56.398305084745765
40.74074074074074 56.398305084745765
40.74074074074074 56.398305084745765
41.25 56.398305084745765
41.25 56.398305084745765
9.675925925925924 38.22033898305085
8.148148148148145 27.966101694915256
18.842592592592588 6.525423728813564
145.64814814814815 -67.58474576271186
150.74074074074076 -71.77966101694915
171.62037037037038 -24.237288135593218
163.9814814814815 -7.457627118644062
149.72222222222223 87.62711864406779
-14.768518518518519 120.25423728813558
-37.17592592592594 137.03389830508473
-43.28703703703704 144.95762711864407
-40.74074074074075 138.89830508474574
12.222222222222221 18.644067796610173
14.259259259259252 18.644067796610173
13.75 13.050847457627121
-10.18518518518519 -51.271186440677965
-35.64814814814815 -86.69491525423729
-40.2314814

In [1]:
from tkinter import *
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk
import tkinter.simpledialog

root = Tk()

#setting up a tkinter canvas
w = Canvas(root, width=500, height=500)
w.pack()

#adding the image
File = askopenfilename(parent=root, initialdir="./",title='Select an image')
original = Image.open(File)
original = original.resize((500,500)) #resize image
img = ImageTk.PhotoImage(original)
w.create_image(0, 0, image=img, anchor="nw")

#ask for pressure and temperature extent
xmt = tkinter.simpledialog.askfloat("Temperature", "degrees in x-axis")
ymp = tkinter.simpledialog.askfloat("Pressure", "bars in y-axis")

#ask for real PT values at origin
xc = tkinter.simpledialog.askfloat("Temperature", "Temperature at origin")
yc = tkinter.simpledialog.askfloat("Pressure", "Pressure at origin")

#instruction on 3 point selection to define grid
tkinter.messagebox.showinfo("Instructions", "Click: \n" 
                                            "1) Origin \n"
                                            "2) Temperature end \n"
                                            "3) Pressure end")

# From here on I have no idea how to get it to work...

# Determine the origin by clicking
def getorigin(eventorigin):
    global x0,y0
    x0 = eventorigin.x
    y0 = eventorigin.y
    print(x0,y0)
    w.bind("<Button 1>",getextentx)
#mouseclick event
w.bind("<Button 1>",getorigin)

# Determine the extent of the figure in the x direction (Temperature)
def getextentx(eventextentx):
    global xe
    xe = eventextentx.x
    print(xe)
    w.bind("<Button 1>",getextenty)

# Determine the extent of the figure in the y direction (Pressure)
def getextenty(eventextenty):
    global ye
    ye = eventextenty.y
    print(ye)
    tkinter.messagebox.showinfo("Grid", "Grid is set. You can start picking coordinates.")
    w.bind("<Button 1>",printcoords)

#Coordinate transformation into Pressure-Temperature space
def printcoords(event):
    xmpx = xe-x0
    xm = xmt/xmpx
    ympx = ye-y0
    ym = -ymp/ympx

    #coordinate transformation
    newx = (event.x-x0)*(xm)+xc
    newy = (event.y-y0)*(ym)+yc

    #outputting x and y coords to console
    print (newx,newy)

root.mainloop()

3 2
322


In [3]:
#Готовый к релизу вариант

from tkinter import *
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk
import tkinter.simpledialog
import math
import numpy as np

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

#Исходные данные

h = 50 #Расстояние от стола до прибора
a = 1.2*h #Длина зоны покрытия
b = math.sqrt((pow(h,2)+pow((a/2),2))) #Второстепенная информация
c = a/2
a = h
angle = (math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))  * 180) / math.pi #угол около камеры
angle = math.ceil(angle)

start_deg = 60

step = angle / (c*2)

discr = angle / 500

print(h,c*2,angle,step,discr)


root = Tk()

#setting up a tkinter canvas
w = Canvas(root, width=500, height=500)
w.pack()

#adding the image
File = askopenfilename(parent=root, initialdir="./",title='Select an image')
original = Image.open(File)
original = original.resize((1000,1000)) #resize image
img = ImageTk.PhotoImage(original)
w.create_image(0, 0, image=img, anchor="nw")


# Determine the origin by clicking
def getorigin(eventorigin):
    global x0,y0
    x0 = eventorigin.x
    y0 = eventorigin.y
    
    print("x - ", x0 - 250 ," y - ", y0 - 250)
    print(cart2pol(x0 - 250 , y0 - 250))
    print()
    dist = math.sqrt((x0 - 250)**2 + (y0 - 250)**2)
    #print("dist - ", dist)
    
    deltaX = x0 - 250;
    deltaY = y0 - 250;
    rad = math.atan2(deltaY, deltaX) # In radians
    deg = rad * (180 / math.pi)
    #print("angle - ", deg)
    print()
    
    #Перевод в новые координаты
    if deg < 0:
        #print("angle - ", deg * -1, "dist - ", dist * -1)
        
        print("Angle rotor 1 - ", start_deg + (dist * -1)*discr)
        print("Angle rotor 2 - ", deg * -1)
        
    else:
        #print("angle - ", deg, "dist - ", dist)
        
        print("Angle rotor 1 - ", start_deg + dist*discr)
        print("Angle rotor 2 - ", deg)

    #print("Angle rotor 1 - ", start_deg + y0*discr)
    w.delete("all")
    w.create_image(0, 0, image=img, anchor="nw")
    w.create_oval(x0-3, y0-3, x0+3, y0+3, width = 0, fill = 'green')
    
#mouseclick event
w.bind("<Button 1>",getorigin)

root.mainloop()

50 60.0 60 1.0 0.12
x -  -7  y -  236
(236.10379073619296, 1.6004486499831696)


Angle rotor 1 -  88.33245488834315
Angle rotor 2 -  91.69895297144595
x -  11  y -  -236
(236.25621684941964, -1.5242198670272982)


Angle rotor 1 -  31.649253978069645
Angle rotor 2 -  87.33136543065574
x -  8  y -  8
(11.313708498984761, 0.7853981633974483)


Angle rotor 1 -  61.35764501987817
Angle rotor 2 -  45.0
x -  0  y -  238
(238.0, 1.5707963267948966)


Angle rotor 1 -  88.56
Angle rotor 2 -  90.0
x -  -26  y -  -14
(29.5296461204668, -2.647651284670212)


Angle rotor 1 -  56.45644246554399
Angle rotor 2 -  151.69924423399362
x -  -14  y -  -15
(20.518284528683193, -2.321725389192837)


Angle rotor 1 -  57.537805856558016
Angle rotor 2 -  133.02506598911802
x -  1  y -  -3
(3.1622776601683795, -1.2490457723982544)


Angle rotor 1 -  59.6205266807798
Angle rotor 2 -  71.56505117707799
x -  1  y -  8
(8.06225774829855, 1.446441332248135)


Angle rotor 1 -  60.96747092979582
Angle rotor 2 -  82.8749

In [42]:
import math

h = 50 #Расстояние от стола до прибора
a = 60 #Длина зоны покрытия
b = math.sqrt((pow(h,2)+pow((a/2),2))) #Второстепенная информация
c = a/2
a = h
angle = (math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))  * 180) / math.pi #угол около камеры




In [22]:
math.tan(h/(a/2))

-10.398778340333516

In [26]:
(h/(a/2) * 180) / math.pi

95.4929658551372

In [27]:
c = math.sqrt((pow(h,2)+pow((a/2),2)))

58.309518948453004

In [39]:
math.cos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))* 180 / math.pi

49.87832345534489

In [34]:
(pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b) #Арк-косинус

0.5144957554275266

In [40]:
math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b)) 

1.0303768265243125

In [41]:
(math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))  * 180) / math.pi #угол около камеры

59.03624346792648

In [44]:
math.ceil(angle)

60

In [56]:

h = 50 #Расстояние от стола до прибора
a = 1.2*h #Длина зоны покрытия
b = math.sqrt((pow(h,2)+pow((a/2),2))) #Второстепенная информация
c = a/2
a = h
angle = (math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))  * 180) / math.pi #угол около камеры
angle = math.ceil(angle)

start_deg = 60

step = angle / (c*2)

discr = angle / 500

print(h,c*2,angle,step,discr)

50 60.0 60 1.0 0.12


In [58]:
60/0.12

500.0

In [60]:
x0*0.12

30.119999999999997

In [61]:
x0*discr

30.119999999999997

In [ ]:
# Import libraries
import RPi.GPIO as GPIO
import time

# Set GPIO numbering mode
GPIO.setmode(GPIO.BOARD)

# Set pin 11 as an output, and define as servo1 as PWM pin
GPIO.setup(11,GPIO.OUT)
servo1 = GPIO.PWM(11,50) # pin 11 for servo1, pulse 50Hz
GPIO.setup(12,GPIO.OUT)
servo2 = GPIO.PWM(12,50)

# Start PWM running, with value of 0 (pulse off)
servo1.start(0)
servo2.start(0)
# Loop to allow user to set servo angle. Try/finally allows exit
# with execution of servo.stop and GPIO cleanup :)

try:
    while True:
        #Ask user for angle and turn servo to it
        i = int(input('Enter servo number: '))
        if i == 1:
            angle = float(input('Enter angle between 0 & 180: '))
            servo1.ChangeDutyCycle(2+(angle/18))
            time.sleep(0.5)
            servo1.ChangeDutyCycle(0)
        if i ==2:
            angle = float(input('Enter angle between 0 & 180: '))
            servo2.ChangeDutyCycle(2+(angle/18))
            time.sleep(0.5)
            servo2.ChangeDutyCycle(0)

finally:
    #Clean things up at the end
    servo1.stop()
    GPIO.cleanup()
    print("Goodbye!")





In [ ]:
#Готовый к релизу вариант

from tkinter import *
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageTk
import tkinter.simpledialog
import math
import numpy as np
import RPi.GPIO as GPIO
import time

# Set GPIO numbering mode
GPIO.setmode(GPIO.BOARD)

# Set pin 11 as an output, and define as servo1 as PWM pin
GPIO.setup(11,GPIO.OUT)
servo1 = GPIO.PWM(11,50) # pin 11 for servo1, pulse 50Hz
GPIO.setup(12,GPIO.OUT)
servo2 = GPIO.PWM(12,50)

# Start PWM running, with value of 0 (pulse off)
servo1.start(0)
servo2.start(0)
# Loop to allow user to set servo angle. Try/finally allows exit
# with execution of servo.stop and GPIO cleanup :)


def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

#Исходные данные

h = 50 #Расстояние от стола до прибора
a = 1.2*h #Длина зоны покрытия
b = math.sqrt((pow(h,2)+pow((a/2),2))) #Второстепенная информация
c = a/2
a = h
angle = (math.acos((pow(b,2)+pow(c,2)-pow(a,2))/(2*c*b))  * 180) / math.pi #угол около камеры
angle = math.ceil(angle)

start_deg = 60

step = angle / (c*2)

discr = angle / 500

print(h,c*2,angle,step,discr)


root = Tk()

#setting up a tkinter canvas
w = Canvas(root, width=500, height=500)
w.pack()

#adding the image
File = askopenfilename(parent=root, initialdir="./",title='Select an image')
original = Image.open(File)
original = original.resize((1000,1000)) #resize image
img = ImageTk.PhotoImage(original)
w.create_image(0, 0, image=img, anchor="nw")


# Determine the origin by clicking
def getorigin(eventorigin):
    global x0,y0
    x0 = eventorigin.x
    y0 = eventorigin.y
    
    print("x - ", x0 - 250 ," y - ", y0 - 250)
    print(cart2pol(x0 - 250 , y0 - 250))
    print()
    dist = math.sqrt((x0 - 250)**2 + (y0 - 250)**2)
    #print("dist - ", dist)
    
    deltaX = x0 - 250;
    deltaY = y0 - 250;
    rad = math.atan2(deltaY, deltaX) # In radians
    deg = rad * (180 / math.pi)
    #print("angle - ", deg)
    print()
    
    #Перевод в новые координаты
    if deg < 0:
        #print("angle - ", deg * -1, "dist - ", dist * -1)
        
        print("Angle rotor 1 - ", start_deg + (dist * -1)*discr)
        print("Angle rotor 2 - ", deg * -1)
        
        angle = start_deg + (dist * -1)*discr
        servo1.ChangeDutyCycle(2+(angle/18))
        time.sleep(0.5)
        servo1.ChangeDutyCycle(0)
        
        angle = deg * -1
        servo2.ChangeDutyCycle(2+(angle/18))
        time.sleep(0.5)
        servo2.ChangeDutyCycle(0)
        
    else:
        #print("angle - ", deg, "dist - ", dist)
        
        print("Angle rotor 1 - ", start_deg + dist*discr)
        print("Angle rotor 2 - ", deg)
        
        angle = start_deg + (dist * -1)*discr
        servo1.ChangeDutyCycle(2+(angle/18))
        time.sleep(0.5)
        servo1.ChangeDutyCycle(0)
        
        angle = deg * -1
        servo2.ChangeDutyCycle(2+(angle/18))
        time.sleep(0.5)
        servo2.ChangeDutyCycle(0)
        

    #print("Angle rotor 1 - ", start_deg + y0*discr)
    w.delete("all")
    w.create_image(0, 0, image=img, anchor="nw")
    w.create_oval(x0-3, y0-3, x0+3, y0+3, width = 0, fill = 'green')
    
#mouseclick event
w.bind("<Button 1>",getorigin)

root.mainloop()
servo1.stop()
servo2.stop()
GPIO.cleanup()
print("Goodbye!")
